<a href="https://colab.research.google.com/github/rujhannajib/mlbNittanyAI-rujhan/blob/main/Rating_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Project: Fine tune an LLM to be an expert in a certain task

- Objective: Fine-tune an LLM to be an expert in classifying online reviews.
- Model: DistilBERT
- Dataset: Yelp Review Full
- Task: Classification
- Library: HuggingFace

By:
- Name: Muhammad Rujhan Najib Bin Fauzi Najib
- PSU ID: 950118053


In [1]:
# installing datasets library from huggingface
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# loading dataset from huggingface
ds = load_dataset("Yelp/yelp_review_full")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [5]:
from datasets import DatasetDict

# Split the test set from the eval set (50% test, 50% eval from the remaining 20%)
test_eval_split = ds['test'].train_test_split(test_size=0.5)

# Create a new DatasetDict with train, test, and validation
ds2 = DatasetDict({
    'train': ds['train'].select(range(50000)),
    'test': test_eval_split['test'].select(range(12000)),
    'eval': test_eval_split['train'].select(range(12000)) # What's left is for evaluation
})

# Checking the resulting splits
ds2

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 12000
    })
    eval: Dataset({
        features: ['label', 'text'],
        num_rows: 12000
    })
})

In [6]:
ds2["train"][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [7]:
len(ds2["train"][4]["text"])

1143

In [8]:
from transformers import DataCollatorWithPadding

# data preparation, tokenizing the text
def tokenize_function(entry):
  return tokenizer(entry['text'], truncation=True, max_length=128)

tokenized_datasets = ds2.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 12000
    })
    eval: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 12000
    })
})

In [10]:
from transformers import TrainingArguments, Trainer
import torch

training_args = TrainingArguments("test-trainer",
                                  report_to="none",
                                  learning_rate = 1e-5,
                                  )


model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english", num_labels=5, ignore_mismatched_sizes=True)

# using GPU for better performance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# using trainer API
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
trainer.train()

Step,Training Loss
500,1.233500
1000,1.092800
1500,1.048300
2000,1.032600
2500,1.010800
3000,1.010400
3500,0.985000
4000,0.979100
4500,0.966100
5000,0.965700


TrainOutput(global_step=18750, training_loss=0.8692266975911458, metrics={'train_runtime': 2445.1762, 'train_samples_per_second': 61.345, 'train_steps_per_second': 7.668, 'total_flos': 4965501821379120.0, 'train_loss': 0.8692266975911458, 'epoch': 3.0})

In [12]:
# making prediction
predictions = trainer.predict(tokenized_datasets["eval"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(12000, 5) (12000,)


In [13]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [14]:
preds, len(preds)

(array([0, 3, 0, ..., 4, 3, 4]), 12000)

In [15]:
len(ds2["eval"]["label"])

12000

In [16]:
# score the model
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_true = ds2["eval"]["label"]
y_pred = preds

acc = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
acc = accuracy_score(y_true, y_pred)

print(f"Accuracy: {acc}")
print(f"F1 Macro: {f1_macro}")
print(f"F1 Micro: {f1_micro}")

Accuracy: 0.5966666666666667
F1 Macro: 0.5980250309350847
F1 Micro: 0.5966666666666667


In [20]:
# Lets see how our model perform on the first 10 texts from evaluation dataset
import pandas as pd
view_eval = {
    "label": ds2["eval"]["label"],
    "prediction": preds,
    "text": ds2["eval"]["text"]
}
eval_df = pd.DataFrame(view_eval)

In [31]:
length_each_class = eval_df.groupby("label").count()["text"].to_list()

# for i in range(5):
#   print(f"Accuracy for class {i}",len(eval_df[eval_df["label"] == eval_df["prediction"]][eval_df["label"] == i]) / length_each_class[i] * 100)

for i in range(len(length_each_class)):
    class_filter = (eval_df["label"] == i) & (eval_df["label"] == eval_df["prediction"])
    accuracy = len(eval_df[class_filter]) / length_each_class[i] * 100
    print(f"Accuracy for class {i}: {accuracy}")


Accuracy for class 0: 72.30308219178082
Accuracy for class 1: 56.26291856138901
Accuracy for class 2: 51.72834485563237
Accuracy for class 3: 53.12232677502139
Accuracy for class 4: 65.19607843137256


In [18]:
# Trying with new input

inputs_text = [
    "Amazing experience! The product exceeded my expectations in every way. Great quality and fast delivery. Will definitely recommend to others.",
    "Very good overall, but the packaging could be improved. The product works as advertised, and I'm happy with the purchase.",
    "It’s decent for the price, but I had some issues with setup. Customer support helped, but it took longer than expected.",
    "Not great. It functions, but there were multiple delays in shipping and the item doesn’t quite match the description.",
    "Terrible experience. The product arrived damaged, and customer service was unresponsive. Wouldn’t buy again."
]


inputs = tokenizer(inputs_text, truncation=True, max_length=128, return_tensors="pt", padding=True)

# Move input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform the prediction
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted label (assuming it's a classification task)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=-1)


# Print each predicted rating
for i, rating in enumerate(predicted_class):
    print(f"Review {i+1} - Predicted rating: {rating.item()}")


Review 1 - Predicted rating: 4
Review 2 - Predicted rating: 2
Review 3 - Predicted rating: 2
Review 4 - Predicted rating: 1
Review 5 - Predicted rating: 0
